# Sequential Feature Selector

![mlxtend](https://rasbt.github.io/mlxtend/img/logo.png)

In a nutshell, SFAs remove or add one feature at the time based on the classifier performance until a feature subset of the desired size k is reached. There are 4 different flavors of SFAs available via the SequentialFeatureSelector:

1.    Sequential Forward Selection (SFS)
2.    Sequential Backward Selection (SBS)
3.    Sequential Forward Floating Selection (SFFS)
4.    Sequential Backward Floating Selection (SBFS)

In [27]:
!pip install mlxtend

## Example 1 - A simple Sequential Forward Selection example

Initializing a simple classifier from scikit-learn:

In [0]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target
knn = KNeighborsClassifier(n_neighbors=4)

We start by selection the "best" 3 features from the Iris dataset via Sequential Forward Selection (SFS). Here, we set forward=True and floating=False. By choosing cv=0, we don't perform any cross-validation, therefore, the performance (here: 'accuracy') is computed entirely on the training set.

In [29]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sfs1 = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=False, 
           verbose=2,
           scoring='accuracy',
           cv=0)

sfs1 = sfs1.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2019-06-16 08:43:01] Features: 1/3 -- score: 0.96[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2019-06-16 08:43:01] Features: 2/3 -- score: 0.9733333333333334[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2019-06-16 08:43:01] Features: 3/3 -- score: 0.9733333333333334

In [30]:
sfs1

SequentialFeatureSelector(clone_estimator=True, cv=0,
                          estimator=KNeighborsClassifier(algorithm='auto',
                                                         leaf_size=30,
                                                         metric='minkowski',
                                                         metric_params=None,
                                                         n_jobs=None,
                                                         n_neighbors=4, p=2,
                                                         weights='uniform'),
                          floating=False, forward=True, k_features=3, n_jobs=1,
                          pre_dispatch='2*n_jobs', scoring='accuracy',
                          verbose=2)

Via the subsets_ attribute, we can take a look at the selected feature indices at each step:

In [31]:
sfs1.subsets_

{1: {'avg_score': 0.96,
  'cv_scores': array([0.96]),
  'feature_idx': (3,),
  'feature_names': ('3',)},
 2: {'avg_score': 0.9733333333333334,
  'cv_scores': array([0.97333333]),
  'feature_idx': (2, 3),
  'feature_names': ('2', '3')},
 3: {'avg_score': 0.9733333333333334,
  'cv_scores': array([0.97333333]),
  'feature_idx': (1, 2, 3),
  'feature_names': ('1', '2', '3')}}

Furthermore, we can access the indices of the 3 best features directly via the k_feature_idx_ attribute:

In [32]:
sfs1.k_feature_idx_

(1, 2, 3)

Finally, the prediction score for these 3 features can be accesses via k_score_:

In [33]:
sfs1.k_score_

0.9733333333333334

## Example 2 - Toggling between SFS, SBS, SFFS, and SBFS

Using the forward and floating parameters, we can toggle between SFS, SBS, SFFS, and SBFS as shown below. Note that we are performing (stratified) 4-fold cross-validation for more robust estimates in contrast to Example 1. Via n_jobs=-1, we choose to run the cross-validation on all our available CPU cores.

In [34]:
# Sequential Forward Selection
sfs = SFS(knn, 
          k_features=3, 
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=4,
          n_jobs=-1)
sfs = sfs.fit(X, y)

print('\nSequential Forward Selection (k=3):')
print(sfs.k_feature_idx_)
print('CV Score:')
print(sfs.k_score_)


Sequential Forward Selection (k=3):
(1, 2, 3)
CV Score:
0.9727564102564104


In [35]:
# Sequential Backward Selection
sbs = SFS(knn, 
          k_features=3, 
          forward=False,
          floating=False, 
          scoring='accuracy',
          cv=4,
          n_jobs=-1)
sbs = sbs.fit(X, y)

print('\nSequential Backward Selection (k=3):')
print(sbs.k_feature_idx_)
print('CV Score:')
print(sbs.k_score_)


Sequential Backward Selection (k=3):
(1, 2, 3)
CV Score:
0.9727564102564104


In [36]:
# Sequential Forward Floating Selection
sffs = SFS(knn, 
           k_features=3, 
           forward=True, 
           floating=True, 
           scoring='accuracy',
           cv=4,
           n_jobs=-1)
sffs = sffs.fit(X, y)

print('\nSequential Forward Floating Selection (k=3):')
print(sffs.k_feature_idx_)
print('CV Score:')
print(sffs.k_score_)


Sequential Forward Floating Selection (k=3):
(1, 2, 3)
CV Score:
0.9727564102564104


In [37]:
# Sequential Backward Floating Selection
sbfs = SFS(knn, 
           k_features=3, 
           forward=False, 
           floating=True, 
           scoring='accuracy',
           cv=4,
           n_jobs=-1)
sbfs = sbfs.fit(X, y)

print('\nSequential Backward Floating Selection (k=3):')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)


Sequential Backward Floating Selection (k=3):
(1, 2, 3)
CV Score:
0.9727564102564104


In this simple scenario, selecting the best 3 features out of the 4 available features in the Iris set, we end up with similar results regardless of which sequential selection algorithms we used.

## Example 3 - Visualizing the results in DataFrames

For our convenience, we can visualize the output from the feature selection in a pandas DataFrame format using the get_metric_dict method of the SequentialFeatureSelector object. The columns std_dev and std_err represent the standard deviation and standard errors of the cross-validation scores, respectively.

Below, we see the DataFrame of the Sequential Forward Selector from Example 2:

In [38]:
import pandas as pd

pd.DataFrame.from_dict(sfs.get_metric_dict()).T

,avg_score,ci_bound,cv_scores,feature_idx,feature_names,std_dev,std_err
1,0.952991,0.0660624,"[0.9743589743589743, 0.9487179487179487, 0.888...","(3,)","(3,)",0.0412122,0.0237939
2,0.959936,0.0494801,"[0.9743589743589743, 0.9487179487179487, 0.916...","(2, 3)","(2, 3)",0.0308676,0.0178214
3,0.972756,0.0315204,"[0.9743589743589743, 1.0, 0.9444444444444444, ...","(1, 2, 3)","(1, 2, 3)",0.0196636,0.0113528


Now, let's compare it to the Sequential Backward Selector:

In [39]:
pd.DataFrame.from_dict(sbs.get_metric_dict()).T

,avg_score,ci_bound,cv_scores,feature_idx,feature_names,std_dev,std_err
4,0.952991,0.0372857,"[0.9743589743589743, 0.9487179487179487, 0.916...","(0, 1, 2, 3)","(0, 1, 2, 3)",0.0232602,0.0134293
3,0.972756,0.0315204,"[0.9743589743589743, 1.0, 0.9444444444444444, ...","(1, 2, 3)","(1, 2, 3)",0.0196636,0.0113528


he ci_bound column in the DataFrames above represents the confidence interval around the computed cross-validation scores. By default, a confidence interval of 95% is used, but we can use different confidence bounds via the confidence_interval parameter. E.g., the confidence bounds for a 90% confidence interval can be obtained as follows:

In [40]:
pd.DataFrame.from_dict(sbs.get_metric_dict(confidence_interval=0.90)).T

,avg_score,ci_bound,cv_scores,feature_idx,feature_names,std_dev,std_err
4,0.952991,0.0286292,"[0.9743589743589743, 0.9487179487179487, 0.916...","(0, 1, 2, 3)","(0, 1, 2, 3)",0.0232602,0.0134293
3,0.972756,0.0242024,"[0.9743589743589743, 1.0, 0.9444444444444444, ...","(1, 2, 3)","(1, 2, 3)",0.0196636,0.0113528


## Example 4 - Sequential Feature Selection for Regression

The boston house-prices dataset (regression).

*    Samples total	506
*    Dimensionality	13
*    Features	real, positive
*   Targets	real 5. - 50.

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split


boston = load_boston()
X, y = boston.data, boston.target

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.2, random_state=1)

lr = LinearRegression()

sfs = SFS(lr, 
          k_features=13, 
          forward=True,
          floating=True,
          scoring='neg_mean_squared_error',
          cv=10)

sfs = sfs.fit(X_train, y_train)

In [42]:
pd.DataFrame.from_dict(sfs.get_metric_dict(confidence_interval=0.95)).T

,avg_score,ci_bound,cv_scores,feature_idx,feature_names,std_dev,std_err
1,-36.8741,8.54624,"[-37.39370891603835, -26.106793116616036, -52....","(12,)","(12,)",11.5068,3.8356
2,-31.2016,7.57522,"[-25.169804795368215, -19.088087851976034, -46...","(10, 12)","(10, 12)",10.1994,3.3998
3,-27.5378,6.93604,"[-16.336970355721064, -24.072760824648345, -44...","(5, 10, 12)","(5, 10, 12)",9.33879,3.11293
4,-27.0367,6.3288,"[-15.438660721234962, -22.24313183403954, -42....","(5, 7, 10, 12)","(5, 7, 10, 12)",8.5212,2.8404
5,-25.7331,7.07646,"[-14.490049455215399, -21.487007196383747, -40...","(4, 5, 7, 10, 12)","(4, 5, 7, 10, 12)",9.52785,3.17595
6,-25.2893,7.05967,"[-13.865379920364296, -22.233971517608712, -39...","(1, 4, 5, 7, 10, 12)","(1, 4, 5, 7, 10, 12)",9.50525,3.16842
7,-24.9711,7.55988,"[-11.693110653303322, -22.79333590086548, -39....","(0, 1, 4, 5, 7, 10, 12)","(0, 1, 4, 5, 7, 10, 12)",10.1787,3.39291
8,-24.7583,6.82365,"[-12.309111029409612, -22.22292283839835, -38....","(0, 1, 4, 5, 7, 8, 10, 12)","(0, 1, 4, 5, 7, 8, 10, 12)",9.18746,3.06249
9,-24.2612,6.95121,"[-11.059484770784369, -22.609412218760056, -37...","(0, 1, 4, 5, 7, 8, 9, 10, 12)","(0, 1, 4, 5, 7, 8, 9, 10, 12)",9.35922,3.11974
10,-23.9598,7.20944,"[-9.874690203063931, -23.026306996614476, -39....","(0, 1, 4, 5, 7, 8, 9, 10, 11, 12)","(0, 1, 4, 5, 7, 8, 9, 10, 11, 12)",9.70689,3.23563


## Example 5 -- Using the Selected Feature Subset For Making New Predictions

In [0]:
df = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
df.sort_values('avg_score', ascending = False, inplace=True)
feature_idx = df['feature_idx'].iloc[0]

In [44]:
print('Selected features:', feature_idx)

Selected features: (0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12)


In [45]:
from sklearn.metrics import mean_squared_error

# Generate the new subsets based on the selected features

X_train_sfs = X_train[:,feature_idx]
X_test_sfs = X_test[:,feature_idx]

# Fit the estimator using the new feature subset
# and make a prediction on the test data
lr.fit(X_train_sfs, y_train)
y_pred = lr.predict(X_test_sfs)

# Compute the accuracy of the prediction
mse = mean_squared_error(y_test, y_pred)
print('Test mean square error: %.2f' % (mse))

Test mean square error: 23.36


## Example 6 -- Sequential Feature Selection and GridSearch (Optional)

In [0]:
# Initialize the dataset

from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.33, random_state=1)

Use scikit-learn's GridSearch to tune the hyperparameters inside and outside the SequentialFeatureSelector:

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import mlxtend

knn = KNeighborsClassifier(n_neighbors=2)

sfs1 = SFS(estimator=knn, 
           k_features=3,
           forward=True, 
           floating=False, 
           scoring='accuracy',
           cv=5)

pipe = Pipeline([('sfs', sfs1), 
                 ('knn', knn)])

param_grid = [
  {'sfs__k_features': [1, 2, 3, 4],
   'sfs__estimator__n_neighbors': [1, 2, 3, 4]}
  ]

gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5,  
                  refit=False)

# run gridearch
gs = gs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


... and the "best" parameters determined by GridSearch are ...

In [48]:
print("Best parameters via GridSearch", gs.best_params_)

Best parameters via GridSearch {'sfs__estimator__n_neighbors': 1, 'sfs__k_features': 3}


If we are interested in the best k feature indices via SequentialFeatureSelection.k_feature_idx_, we have to initialize a GridSearchCV object with refit=True. Now, the grid search object will take the complete training dataset and the best parameters, which it found via cross-validation, to train the estimator pipeline.

In [49]:
gs = GridSearchCV(estimator=pipe, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  n_jobs=1, 
                  cv=5, 
                  refit=True)
gs = gs.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


After running the grid search, we can access the individual pipeline objects of the best_estimator_ via the steps attribute.

Via sub-indexing, we can then obtain the best-selected feature subset:

In [50]:
print('Best features:', gs.best_estimator_.steps[0][1].k_feature_idx_)

Best features: (0, 1, 3)


During cross-validation, this feature combination had a CV accuracy of:

In [51]:
print('Best score:', gs.best_score_)

Best score: 0.94


In [52]:
gs.best_params_


{'sfs__estimator__n_neighbors': 1, 'sfs__k_features': 3}